In [57]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
rectangles = []

In [58]:
def find_txt_information_referee(class_id, x_min, y_min, x_max, y_max, width, height):
    x_center = ((x_min + x_max) / 2.0) / width
    y_center = ((y_min + y_max) / 2.0) / height

    resized_width = (x_max - x_min) / width
    resized_height = (y_max - y_min) / height

    rectangles.append([class_id, x_center, y_center, resized_width, resized_height])

In [59]:
def save_yolo_txt(image_name, path_name, rectangles):
    if not os.path.exists(path_name):
        os.makedirs(path_name, exist_ok=True)
    txt_path = image_name.replace("images", "labels").replace("jpg", "txt")
    
    with open(txt_path, "w") as f:
        for rectangle in rectangles:
            class_id, x_center, y_center, width, height = rectangle
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [ ]:
drawing = False
ix = -1
iy = -1
class_id = 0
number = False
def draw_rectangle_and_create_txt_file(event, x, y, flags, param):
    global drawing, img, ix, iy, class_id, number
    if event == cv2.EVENT_RBUTTONDOWN:
        number = True
    elif event == cv2.EVENT_RBUTTONUP:
        number = False
        class_id += 1
        
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x,y), color = (0, 255, 0), thickness= 1)
        find_txt_information_referee(class_id, ix, iy, x, y, 800, 600)
        save_yolo_txt("tools/images/val/frame_4.jpg", "tools/labels/val", rectangles)

In [115]:
img = cv2.imread('tools/images/val/frame_4.jpg', cv2.IMREAD_COLOR_BGR)
cv2.namedWindow("new_images")
cv2.setMouseCallback("new_images",draw_rectangle_and_create_txt_file)

while True:
    cv2.imshow("new_images", img)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

---

In [138]:
model = YOLO("yolov8n.pt")

In [139]:
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model.train(data= "data.yaml", epochs = 200, batch = 16, imgsz = 640, save = True, device = device)
video_path = "video.mp4"

capture = cv2.VideoCapture(video_path)

fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "output_video.mp4"
output = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        break
    results = model(frame)
    annotated_frame = results[0].plot()
    output.write(annotated_frame)
    cv2.imshow("YOLO Video Object Detection", annotated_frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

capture.release()
output.release()
cv2.destroyAllWindows()